## Data pre-processing

Dependencies

In [2]:
import os
import pdfplumber
import re
import pandas as pd
import pickle

### Add columns to dataframe

In [35]:
folder_path = "../raw-dataset/"

policy_titles = []
policy_codes = []
article_names = []
article_content = []

def extract_title(pdf):
    first_page = pdf.pages[0]
    page_text = first_page.extract_text()
    lines = page_text.split('\n')
    title_lines = []

    for line in lines:
        if line.isupper():
            title_lines.append(line.strip())
        elif title_lines:
            break
    
    title = ' '.join(title_lines).strip() if title_lines else None
    
    return title


for file in os.listdir(folder_path):
    if file.endswith(".pdf"):
        file_path = os.path.join(folder_path, file)

        with pdfplumber.open(file_path) as pdf:
            policy_title = extract_title(pdf)

            for page in pdf.pages:
                page_text = page.extract_text()
                page_text = re.sub(r'Artículo', 'ARTÍCULO', page_text)
                articles = []
                articles += page_text.split("ARTÍCULO")[1:]
                
                for article in articles:
                    split_article = article.split("\n", 1)
                    if len(split_article) == 2:
                        article_name, content = split_article
                    else:
                        article_name = "Nombre del artículo no encontrado"
                        content = article

                    policy_titles.append(policy_title)
                    policy_codes.append(file.replace(".pdf", ""))
                    article_names.append(article_name.strip())
                    article_content.append(content.strip())


df = pd.DataFrame({
    "Policy_Title": policy_titles,
    "Policy_Code": policy_codes,
    "Article_Name": article_names,
    "Article_Content": article_content,
})

result = df.loc[df['Policy_Code'] == 'POL320210063.pdf']

display(df)

,Policy_Title,Policy_Code,Article_Name,Article_Content
0,PÓLIZA DE SEGURO PARA PRESTACIONES MÉDICAS DER...,POL320180100,1°: REGLAS APLICABLES AL CONTRATO,Se aplicarán al presente contrato de seguro la...
1,PÓLIZA DE SEGURO PARA PRESTACIONES MÉDICAS DER...,POL320180100,2º: COBERTURA Y MATERIA ASEGURADA,La Compañía Aseguradora reembolsará al asegura...
2,PÓLIZA DE SEGURO PARA PRESTACIONES MÉDICAS DER...,POL320180100,5° de estas,Condiciones Generales; y 6) Que las prestacion...
3,PÓLIZA DE SEGURO PARA PRESTACIONES MÉDICAS DER...,POL320180100,"4°, numeral 17 de estas Condiciones",Generales y establecido en las Condiciones par...
4,PÓLIZA DE SEGURO PARA PRESTACIONES MÉDICAS DER...,POL320180100,"4°, numeral 28 de estas Condiciones",Generales y cuya extensión se determina en las...
...,...,...,...,...
260,SEGURO INDIVIDUAL CATASTRÓFICO POR EVENTO,POL320200071,15. MONEDA O UNIDAD DEL CONTRATO DE SEGURO.,"Los montos asegurados, los valores y las prima..."
261,SEGURO INDIVIDUAL CATASTRÓFICO POR EVENTO,POL320200071,16. COMUNICACION ENTRE LAS PARTES.,"Cualquier comunicación, declaración o notifica..."
262,SEGURO INDIVIDUAL CATASTRÓFICO POR EVENTO,POL320200071,18. DERECHO DE RETRACTO.,De conformidad al artículo 538 del Código de C...
263,SEGURO INDIVIDUAL CATASTRÓFICO POR EVENTO,POL320200071,19. DOMICILIO.,Para todos los efectos legales del presente co...


Save Dataframe 

In [ ]:
# Guardar el DataFrame en un archivo pickle
pickle.dump(df, open('df.pickle', 'wb'))

# Abrir el archivo pickle y cargar el DataFrame
loaded_df = pickle.load(open('df.pickle', 'rb'))

# Mostrar el DataFrame cargado
print(loaded_df)